In [3]:
# === P-TLI — Threshold por monto (Inbound Cash) ================================
# Regla: When an Inbound Cash transaction is over [Amount] {var.Amount} CLP, apply {action}.
# Justificación: seleccionar el monto en el percentil 95 de la distribución.

import pandas as pd
import numpy as np

# -------- Parámetros editables --------
PATH = "../../data/tx/transacciones_cash_2025__with_subsub.csv"
SUBSUBSEGMENTS = "I-2"         # <-- ajusta el sub-subsegmento
PCTS = [90, 95, 97, 99]         # percentiles a reportar

# -------- Carga csv --------
df = pd.read_csv(PATH, dtype={"customer_id": "string"}, encoding="utf-8-sig")

# Filtrado por sub-subsegmento
if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS))   

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

# -------- Filtro según regla --------
df["tx_base_amount"] = pd.to_numeric(df["tx_base_amount"], errors="coerce")
mask = (
    (df["tx_direction"].astype(str).str.title() == "Inbound") &
    (df["tx_type"].astype(str).str.title() == "Cash") &
    (df["tx_base_amount"] > 0)  # montos válidos y positivos en CLP
)
g = df.loc[mask, ["tx_base_amount"]].dropna()

if g.empty:
    print("No hay transacciones elegibles para P-TLI con los filtros dados.")
else:
    s = g["tx_base_amount"].astype(float)
    stats = {f"p{p}": float(np.percentile(s, p)) for p in PCTS}

    # Recomendado:p95
    recommended_amount = int(round(stats["p95"]))

    print("=== P-TLI — Percentiles de monto (CLP, Inbound Cash) ===")
    for p in PCTS:
        v = stats[f"p{p}"]
        print(f"p{p:>2}: {v:,.0f}")
    print(f"\nAmount recomendado (p95): {recommended_amount:,.0f} CLP")


C:\Users\vinso\AppData\Local\Temp\ipykernel_48912\2773610635.py:14: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, dtype={"customer_id": "string"}, encoding="utf-8-sig")


=== P-TLI — Percentiles de monto (CLP, Inbound Cash) ===
p90: 1,820,083,200
p95: 2,402,783,176
p97: 2,915,963,280
p99: 4,021,784,704

Amount recomendado (p95): 2,402,783,176 CLP


# Simulación alertas

In [5]:
# === P-TLI — Sensibilidad (Actual vs propuestos) ===============================
# LÓGICA EXACTA:
# tx_direction = Inbound
# AND tx_base_amount > [Amount]
# Unidad = transacciones que cumplen

import pandas as pd
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

PATH="../../data/tx_retail_whale.csv"
PARAMS={
   # "Actual":{"Amount":200_000_000},
   # "p90":   {"Amount":254_400_000},
    "p95":   {"Amount":617_000_000},
    "p97":   {"Amount":873_000_873},
    "p99":   {"Amount":1_096_000_000},
}

df=pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_direction"]=df["tx_direction"].astype(str).str.title()
df["tx_base_amount"]=pd.to_numeric(df["tx_base_amount"], errors="coerce")

g=df[(df["tx_direction"].eq("Inbound")) & df["tx_base_amount"].notna()].copy()
param_tbl=pd.DataFrame(PARAMS).T.rename_axis("escenario").reset_index()
print("=== P-TLI — Parámetros (Amount) ==="); display(param_tbl)

counts={k:int((g["tx_base_amount"]>v["Amount"]).sum()) for k,v in PARAMS.items()}
out=pd.DataFrame([{
    "alertas_actual":counts.get("Actual",0),
    "alertas_p90":counts.get("p90",0),
    "alertas_p95":counts.get("p95",0),
    "alertas_p97":counts.get("p97",0),
    "alertas_p99":counts.get("p99",0),
}])
print("=== P-TLI — Alertas por escenario (tx) ==="); display(out)


=== P-TLI — Parámetros (Amount) ===


,escenario,Amount
0,p95,617000000
1,p97,873000873
2,p99,1096000000


=== P-TLI — Alertas por escenario (tx) ===


,alertas_actual,alertas_p90,alertas_p95,alertas_p97,alertas_p99
0,0,0,14,9,3
